# Set up

In [2]:
!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_lg
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
!pip install metaphone

     |████████████████████████████████| 96.4 MB 1.2 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051302 sha256=303849018b64792d2d1508e3d0e5d7e495c3a48f901a62e8b9e404334ccf11c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-a7pvcr37/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
     |████████████████████████████████| 126 kB 5.2 MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52578 sha256=015dcdde0dca95b3a10b4e637dd30d9ab75fa5ad59fe1f82a4bb6539b9d2d0f4
  Stored in directory: /root/.cache/pip/wheels/09/bc/67/75c9de8e9726460bc0b101ad225ad025cb8ce9e0759beb9d52
Successfully built polyglot
     |████████████████████████████████| 299 kB 5.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing whee

In [3]:
import pandas as pd
import numpy as np
import os
import math
from google.colab import drive
import polyglot
import spacy
import nltk
import re

from nltk.tokenize import TweetTokenizer
from polyglot.text import Text, Word

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
import en_core_web_md
#import en_core_web_lg
from dateutil import parser
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score, recall_score,precision_score,  classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from collections import Counter

In [4]:
drive.mount('/content/drive', force_remount= True)
#os.chdir('/content/drive/MyDrive/Asesoftware/Plan semilla/NLP Challenge') 
os.chdir("/content/drive/MyDrive/NLP Challenge") #Thomas
#os.chdir('/content/drive/MyDrive/ASESOFTWARE/NLP/NLP Challenge') #Sebastian
os.listdir("./")

Mounted at /content/drive


['data',
 'Thomas Edition IR Classiffication.ipynb',
 'results',
 'IR Classiffication.ipynb',
 'NN.ipynb',
 'Logistic Regression FTW.ipynb']

In [5]:
DATA_PATH = './data/'
#RESULTS_PATH = './results/'

train = pd.read_csv(DATA_PATH+'train.csv', encoding='utf-8')
test = pd.read_csv(DATA_PATH+'test.csv')
train.shape, test.shape

((7613, 5), (3263, 4))

In [6]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [7]:
test.text.nunique()

3243

In [8]:
train = train.drop_duplicates(subset = ['text'])
print(train.shape)

X = train.drop('target',axis=1)
y = train.target
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

(7503, 5)


In [9]:
train['target'].value_counts()

0    4305
1    3198
Name: target, dtype: int64

In [10]:
train[train['target']==1].text.tolist()

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 'Forest fire near La Ronge Sask. Canada',
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
 '13,000 people receive #wildfires evacuation orders in California ',
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods...",
 "There's an emergency evacuation happening now in the building across the street",
 "I'm afraid that the tornado is coming to our area...",
 'Three people died from the heat wave so far',
 'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA 

In [11]:
train[train['target']==0].text.tolist()

["What's up man?",
 'I love fruits',
 'Summer is lovely',
 'My car is so fast',
 'What a goooooooaaaaaal!!!!!!',
 'this is ridiculous....',
 'London is cool ;)',
 'Love skiing',
 'What a wonderful day!',
 'LOOOOOOL',
 "No way...I can't eat that shit",
 'Was in NYC last week!',
 'Love my girlfriend',
 'Cooool :)',
 'Do you like pasta?',
 'The end!',
 'We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw',
 'Crying out for more! Set me ablaze',
 'On plus side LOOK AT THE SKY LAST NIGHT IT WAS ABLAZE http://t.co/qqsmshaJ3N',
 "@PhDSquares #mufc they've built so much hype around new acquisitions but I doubt they will set the EPL ablaze this season.",
 'Ablaze for you Lord :D',
 'Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw',
 "on the outside you're ablaze and alive\nbut you're dead inside",
 'Had an awesome time visiting the CFC head office the ancop site and ablaze. Thanks to Tita Vida for taking care of 

All the tweets are in english?  RTA: ....


# Modelo

In [12]:
X_train = pd.read_csv('./results/X_train.csv')
y_train = pd.read_csv('./results/y_train.csv')
X_test = pd.read_csv('./results/X_test.csv')
y_test = pd.read_csv('./results/y_test.csv')

In [21]:
from sklearn.linear_model import LogisticRegression #the model we're using 

model = LogisticRegression(penalty='l1',multi_class='ovr', solver='saga', max_iter= 1000,class_weight = 'balanced',random_state =1234)
model.fit(X_train, y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(class_weight='balanced', max_iter=1000, multi_class='ovr',
                   penalty='l1', random_state=1234, solver='saga')

In [23]:
# Test our data

#score = round(model.score(X_test, y_test)*100,2)

#print(score)


pred = model.predict(X_test)
print('----------Model report on all classes ----------')
print(classification_report(y_test,pred))

----------Model report on all classes ----------
              precision    recall  f1-score   support

           0       0.57      0.84      0.68       874
           1       0.37      0.13      0.19       627

    accuracy                           0.54      1501
   macro avg       0.47      0.49      0.44      1501
weighted avg       0.49      0.54      0.48      1501



/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [20]:
#gridsearch


param_grid = [ #A dictionary containing all the HP options
  {'penalty': ['l2', ' none'], #type of regularization, l1 is lasso, l2 is ridge, elasticnet is both
    'C': np.logspace(-2,2,5),  #inverse of regularization strength (i.e. 1 /lambda)
    'solver' : ['newton-cg','sag','saga'], # differnt solvers
    'max_iter': [300] #if more iterations are needed
  }

]

clf = GridSearchCV(model, param_grid = param_grid, cv = 3, verbose = 1, n_jobs=1)

In [ ]:
best_clf = clf.fit(X_train,y_train)

Fitting 3 folds for each of 80 candidates, totalling 240 fits


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [ ]:
pred = best_clf.predict(X_test)
print('----------Model report on all classes ----------')
print(classification_report(y_test,pred))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


----------Model report on all classes ----------
              precision    recall  f1-score   support

           0       0.57      0.73      0.64       874
           1       0.37      0.22      0.28       627

    accuracy                           0.52      1501
   macro avg       0.47      0.48      0.46      1501
weighted avg       0.49      0.52      0.49      1501

